In [ ]:
SELECT DATE_FORMAT(trans_date,'%Y-%m') AS month,
country,
SUM(CASE WHEN state = 'approved' THEN 1 ELSE 0 END) AS approved_count,
SUM(CASE WHEN state = 'approved' THEN amount ELSE 0 END) AS approved_amount,
SUM(CASE WHEN state = '0' THEN 1 ELSE 0 END) AS chargeback_count,
SUM(CASE WHEN state = '0' THEN amount ELSE 0 END) AS chargeback_amount
FROM (
SELECT id, country, amount,state, trans_date
FROM Transactions T
WHERE state = 'approved'
UNION ALL
SELECT C.trans_id,T1.country,T1.amount,0, C.trans_date
FROM Chargebacks C JOIN Transactions T1 ON C.trans_id = T1.id) tbl
GROUP BY month,country
;
Runtime: 291 ms, faster than 57.43% of MySQL online submissions for Monthly Transactions II.
Memory Usage: 0B, less than 100.00% of MySQL online submissions for Monthly Transactions II.







In [ ]:
tried this version but did not work
# select left(trans_date, 7) as month, country, count(*) as approved_count, amount as approved_amount, (select count(*) from chargebacks c join transactions t on t.id=c.trans_id) as chargeback_count
# from transactions
# group by month, country;


In [ ]:
this works except for the glitch but this does not have the date:  2019-09-18 in chargebacks table

    select left(c.trans_date, 7) as month, count(*) as approved_count, sum(case when left(t.trans_date, 7)=left(c.trans_date, 7) then amount else 0 end) as approved_amount from transactions t
join chargebacks c
on left(t.trans_date, 7)=left(c.trans_date, 7)
where t.state="approved"
group by left(c.trans_date, 7);



In [ ]:
select t. month
      ,t.country
      ,sum(t.approved_count) as approved_count
      ,sum(t.approved_amount) as approved_amount
      ,sum(t.chargeback_count) as chargeback_count
      ,sum(t.chargeback_amount) as chargeback_amount
 from (SELECT left(t.trans_date, 7) as month
             ,t.country
             ,sum(amount) as approved_amount
             ,count(*) as approved_count
             ,0 as chargeback_amount
             ,0 as chargeback_count
         from Transactions t
        where t.state = 'approved' 
       #and t.id = 101
      group by left(t.trans_date, 7) 
              ,t.country
UNION ALL 
         SELECT left(c.trans_date, 7)  as month
                ,t.country
                ,0 as approved_amount
                ,0 as approved_count          
                ,sum(t.amount) as chargeback_amount
                ,count(*) as chargeback_count
            from Chargebacks c
      inner join Transactions t
              on t.id = c.trans_id
       #and c.trans_id = 102
        group by left(c.trans_date, 7)  
                ,t.country )t  
group by t.month, t.country

Runtime: 302 ms, faster than 33.43% of MySQL online submissions for Monthly Transactions II.
Memory Usage: 0B, less than 100.00% of MySQL online submissions for Monthly Transactions II.

In [ ]:
Transactions table:
+------+---------+----------+--------+------------+
| id   | country | state    | amount | trans_date |
+------+---------+----------+--------+------------+
| 101  | US      | approved | 1000   | 2019-05-18 |
| 102  | US      | declined | 2000   | 2019-05-19 |
| 103  | US      | approved | 3000   | 2019-06-10 |
| 104  | US      | approved | 4000   | 2019-06-13 |
| 105  | US      | approved | 5000   | 2019-06-15 |
+------+---------+----------+--------+------------+

Chargebacks table:
+------------+------------+
| trans_id   | trans_date |
+------------+------------+
| 102        | 2019-05-29 |
| 101        | 2019-06-30 |
| 105        | 2019-09-18 |
+------------+------------+

In [ ]:
SELECT C.trans_id,T1.country,T1.amount,0, C.trans_date
FROM Chargebacks C JOIN Transactions T1 ON C.trans_id = T1.id;


{"headers": ["trans_id", "country", "amount", "0", "trans_date"], "values": [[101, "US", 1000, 0, "2019-06-30"], [102, "US", 2000, 0, "2019-05-29"], [105, "US", 5000, 0, "2019-09-18"]]}

In [ ]:
SELECT id, country, amount,state, trans_date
FROM Transactions T
WHERE state = 'approved';

glitch: id:104 can not be found in the outcome of this query
{"headers": ["id", "country", "amount", "state", "trans_date"], "values": [[101, "US", 1000, "approved", "2019-05-18"], [103, "US", 3000, "approved", "2019-06-10"], [105, "US", 5000, "approved", "2019-06-15"]]}



In [ ]:
i think there was a glitch on leetcode re the problem. my query worked. I could not print the 0 values for the dates 
that are available in one table but not in the other. but my logic was right. the code did not produce any outcome for
id: 104 in the transactions table. weird!

In [ ]:
SELECT id, country, amount,state, trans_date
FROM Transactions T
WHERE state = 'approved';
UNION ALL
SELECT C.trans_id,T1.country,T1.amount,0, C.trans_date
FROM Chargebacks C JOIN Transactions T1 ON C.trans_id = T1.id;


{"headers": ["id", "country", "amount", "state", "trans_date"], "values": [[101, "US", 1000, "approved", "2019-05-18"], [103, "US", 3000, "approved", "2019-06-10"], [105, "US", 5000, "approved", "2019-06-15"], [101, "US", 1000, "0", "2019-06-30"], [102, "US", 2000, "0", "2019-05-29"], [105, "US", 5000, "0", "2019-09-18"]]}